In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import json
from fastai.vision import *
from fastai.callbacks import *
import torch.nn as nn

In [ ]:
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled, torch.backends.cudnn.deterministic)
torch.backends.cudnn.deterministic = True
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled, torch.backends.cudnn.deterministic)

In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=0, max_warp=0)
tfms

In [ ]:
data_path = "../input/aptos2019-blindness-detection"
train_label_file = "train.csv"
train_images_folder = "train_images"
test_label_file = "test.csv"
test_images_folder = "test_images"
image_suffix = ".png"

In [ ]:
split_pct = 0.1
bs = 64
size = 224

In [ ]:
fname = os.path.join(data_path, train_label_file)

In [ ]:
df = pd.read_csv(fname)
df.head(5)

In [ ]:
_ = df.hist()

In [ ]:
np.random.seed(42)
src = (ImageList.from_csv(data_path, train_label_file, folder=train_images_folder, suffix=image_suffix)
       .split_by_rand_pct(split_pct)
       .label_from_df())
src

In [ ]:
data = (src.transform(tfms, size=size)
        .databunch(bs=bs).normalize())
data.c, data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7, 6))

In [ ]:
arch = models.vgg16_bn

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"

In [ ]:
if not os.path.exists('/tmp/.cache/torch/checkpoints/'):
        os.makedirs('/tmp/.cache/torch/checkpoints/')

In [ ]:
print(os.listdir("/tmp/.cache/torch/checkpoints"))

In [ ]:
! cp "../input/fastai-pretrained-models/vgg16_bn-6c64b313.pth" "/tmp/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth"

In [ ]:
learn = cnn_learner(data, arch, metrics=[error_rate, accuracy, kappa], model_dir= "../saved_models")

In [ ]:
learn.loss_func

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=3., reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = ((1 - pt)**self.gamma) * CE_loss
        if self.reduction == 'sum':
            return F_loss.sum()
        elif self.reduction == 'mean':
            return F_loss.mean()

In [ ]:
learn.loss_func = FocalLoss()
learn.loss_func

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
lr = 1e-2/2

In [ ]:
learn.fit_one_cycle(5, lr,  callbacks=[SaveModelCallback(learn, monitor='kappa_score', mode='max', name="headonly_vgg16_bn")])

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
learn.load('headonly_vgg16_bn');

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(2, slice(1e-4, lr/5), callbacks=[SaveModelCallback(learn, monitor='kappa_score', mode='max', name="best_entirenet_vgg16_bn_224")])

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
learn.load('best_entirenet_vgg16_bn_224');

In [ ]:
sample_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

In [ ]:
test = ImageList.from_df(sample_df, data_path, folder=test_images_folder, suffix=image_suffix)
test

In [ ]:
learn.data.add_test(test, label=test_label_file)

In [ ]:
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
sample_df.diagnosis = preds.argmax(1)
sample_df.head()

In [ ]:
sample_df.to_csv('submission.csv',index=False)
_ = sample_df.hist()

In [ ]:
print(os.listdir("../input/saved_models"))